In [1]:
import os
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit,train_test_split
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import backend as K
from tensorflow import one_hot

### Reading the Training and Test data
The dimention of the data shall be #Trials x #Contatcts

In [2]:
EEG_data = "Data/EEG"
Label_data = "Data/Labels"
# Reading Training EEG Data and their labels
train_data = pd.read_csv(os.path.join(EEG_data,'training_set.csv'), header=None)
train_data = np.array(train_data).astype('float32')

# Reading Test EEG Data and their labels
test_data = pd.read_csv(os.path.join(EEG_data, 'test_set.csv'), header=None)
test_data = np.array(test_data).astype('float32')

In [3]:
train_labels = pd.read_csv(os.path.join(Label_data , 'training_label.csv'), header=None)
train_labels = np.array(train_labels).astype('int')
test_labels = pd.read_csv(os.path.join(Label_data,'test_label.csv'), header=None)
test_labels = np.array(test_labels).astype('int')

Both Training and Test Data have 64x64 feature size which come from 64-contacts EEG signal and 64 sample in time \
which will be used for classification purposes

In [4]:
print('Train Data Size: {}'.format(train_data.shape))
print('Test Data Size: {}'.format(test_data.shape))

Train Data Size: (76356, 4096)
Test Data Size: (8484, 4096)


In [5]:
# One-hot representation of trian and test labels
train_labels = tf.one_hot(indices=train_labels, depth=4)
train_labels = tf.squeeze(train_labels)
test_labels = tf.one_hot(indices=test_labels, depth=4)
test_labels = tf.squeeze(test_labels)

In [11]:
# Model Hyper-parameters
n_contacts   = 64      # The number of EEG contacts at each time point
n_time  = 64      # number of EEG time points to feed to the model
n_neurons_lstm_1 = 128     # number of neurons in the LSTM layer
n_neurons_lstm_2 = 64     # number of neurons in the LSTM layer
n_attention = 8  # The number of neurons in attention layer

n_class   = 4     # The number of classification classes
n_neurons_FC  = 64    # The number of hidden units in the FC layer
n_neurons_SM  = n_class    # The number of hidden units in the SoftMax layer
num_epoch = 300   # The number of Epochs that the Model run
keep_rate = 0.75  # Keep rate of the Dropout
dropout_prob = 1-keep_rate
n_batch = 128


In [7]:
X = tf.reshape(train_data, [-1, n_time, n_contacts])
y = train_labels

In [8]:
class attention(Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(attention,self).__init__()
    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        super(attention,self).build(input_shape)
    def call(self, x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [12]:
model = Sequential()
model.add(Bidirectional(LSTM(n_neurons_lstm_1, return_sequences=True,activation='tanh',recurrent_dropout=dropout_prob),
                        input_shape=(X.shape[1],X.shape[2])))
model.add(LSTM(n_neurons_lstm_2, return_sequences=True,activation='tanh',recurrent_dropout=dropout_prob,
                        input_shape=(X.shape[1],X.shape[2])))

model.add(attention(return_sequences=True)) # receive 3D and output 3D
model.add(TimeDistributed(Dense(n_neurons_FC,activation='relu')))
model.add(TimeDistributed(Dense(n_neurons_SM,activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 64, 256)           197632    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64, 64)            82176     
_________________________________________________________________
attention (attention)        (None, 64, 64)            128       
_________________________________________________________________
time_distributed (TimeDistri (None, 64, 64)            4160      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 64, 4)             260       
Total params: 284,356
Trainable params: 284,356
Non-trainable params: 0
_________________________________________________________________
None


### Training and Testing the Model

In [64]:
history = {}
cv = 10
for j in range(cv): 
    history[j] = []
accuracy = np.zeros((y.shape[1],cv))
SSS = StratifiedShuffleSplit(n_splits=cv, test_size=0.4)
j=0
for train_index, test_index in SSS.split(X, y):
    X_train, X_test = X[train_index,:,:], X[test_index,:,:]
    y_train, y_test = y[train_index,:], y[test_index,:]
    history[j] = model.fit(X_train, one_hot(y_train,depth=3), epochs=n_epoch, batch_size=n_batch,validation_split=0.2 ,verbose=1)
    result = model.predict(X_test, batch_size=n_batch, verbose=0)
    y_classes = tf.argmax(result,2)
    for i in range(y_test.shape[1]):
        accuracy[i,j] =  accuracy_score(y_test[:,i], y_classes[:,i])
    j = j+1